<H2>Welcome. This notebook is used for IBM capstone project. Toronto neighborhood clustering</H2>

<h2>The Notebook is devied into three sections</h2>
<ul>
    <li>1. Scraping data and crating a pandas Dataframe.</li>
    <li>2. Create dataframe with latitude longitude for each postal code.</li>
    <li>3. Cluster the Toronto neighborhoods.</li>
</ul>

<h2><b>----Section1:Scraping data and crating a pandas Dataframe.----</b></h2>

<h3>Section 1 Covers following points of peer graded assignment</h3>
<ul>
<li>1. Start by creating a new Notebook for this assignment.</li>
<li>2. Use the Notebook to build the code to scrape the following Wikipedia page,<br> 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the<br> 
data that is in the table of postal codes and to transform the data into a pandas  dataframe<br> 
like the one shown below:</li>
<li>3. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
<li>4. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
<li>5. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.</li>
</ul>


In [1]:
#install dependencies
import sys
import pandas as pd
import numpy as np

In [2]:
#Use wikipedia url given in the assignment 
scrape_page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
#Use pandas library to read the table from given url
ls_toronto_raw_data = pd.read_html(scrape_page_url)

In [4]:
#get the list data into pandas dataframe
df_toronto_nbh_raw_data = pd.DataFrame(ls_toronto_raw_data[0])

In [5]:
#remove any duplicate values
df_toronto_nbh_raw_data.drop_duplicates(inplace=True)

In [6]:
# This cell covers following points in the 
## The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
## Only process the cells that have an assigned borough. Ignore cells with a borough = Not assigned.

import numpy as np

#get row indexs with Borough value of 'Not assigned'
idx_NotAssigned_Borough = df_toronto_nbh_raw_data[df_toronto_nbh_raw_data['Borough']=='Not assigned'].index.values
df_toronto_nbh_raw_data.drop(index = idx_NotAssigned_Borough, inplace=True)

#get row indexs with neighbourhood value of 'Not assigned'
idx_NotAssigned_Neighbourhood = df_toronto_nbh_raw_data[df_toronto_nbh_raw_data['Neighbourhood']=='Not assigned'].index.values   
    
for idx in idx_NotAssigned_Neighbourhood:
    #assign borough value to neighbourhood if Borough value exists but neighbourhood does not
    df_toronto_nbh_raw_data.iloc[idx][2] = df_toronto_nbh_raw_data.iloc[idx][1]

In [7]:
#This cell removed duplicate entries of postal code and adds one entry with different neighbourhoods seperated by comma in
#neighbourhood column

#get index of duplicate postal codes
indx_duplicate_postcode = df_toronto_nbh_raw_data[df_toronto_nbh_raw_data['Postal Code'].duplicated()].index.values

if(indx_duplicate_postcode.size > 0):
    for indx in indx_duplicate_postcode:
        #get all values in dataframe with same postal code
        postalCode = df_toronto_nbh_raw_data.iloc[indx][0]
        df_duplicate_postCodes = df_toronto_nbh_raw_data[df_toronto_nbh_raw_data['Postal Code']==postalCode]
        if(df_duplicate_postCodes.shape[0]>1):
            iteration_count = 1
            neigh_comma_separated = ''
            boroughVal = ''
            for borough, neighbourhood in zip(df_duplicate_postCodes['Borough'], df_duplicate_postCodes['Neighbourhood']):
                 commaVal = ',' if iteration_count<df_duplicate_postCodes.shape[0] else ''
                 
                 #neighbourhoods are arranged as a comman separated string.
                 neigh_comma_separated = neigh_comma_separated + neighbourhood + commaVal
                 boroughVal = borough
                 iteration_count+= 1
            
            #Old rows with repeated postal code are removed
            df_toronto_nbh_raw_data.drop(index=df_toronto_nbh_raw_data[df_toronto_nbh_raw_data['Postal Code']==postalCode].index.values, inplace=True)
            tempDict = [{'Postal Code':postalCode,'Borough':boroughVal,'Neighbourhood':neigh_comma_separated}]
            deduplicated_postalCode = pd.DataFrame(tempDict)
            #New row with one postal code and multiple neighbourhoods is added to the dataframe.
            df_toronto_nbh_raw_data = df_toronto_nbh_raw_data.append(deduplicated_postalCode)
            df_toronto_nbh_raw_data.reset_index(inplace = True)

In [8]:
#reindex the dataframe
print('The number of rows in the dataframe are ',df_toronto_nbh_raw_data.shape[0])

The number of rows in the dataframe are  103


<hr><h2>End of section1</h2><hr>

<h2><b>----Section 2: Scraping data and crating a pandas Dataframe.----</b></h2>

<h3>Section 2 Covers following points of peer graded assignment</h3>
<ul>
<li>1. Read CSV file given by URL http://cocl.us/Geospatial_data </li>
<li>2. Map each postal code to its respective latitude and longitute values</li>
</ul>


In [9]:
#The API could not get the lat and lng for all the postal codes thus using the csv file given in the assignment

#create pandas dataframe out of CSV file given at the URL location http://cocl.us/Geospatial_data
df_lat_lng =  pd.read_csv('http://cocl.us/Geospatial_data')
df_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Merge 'df_toronto_nbh_raw_data' and 'df_lat_lng da' dataframes, to map postal codes to respective latitude longitude values.
df_toronto_nbh_data = df_toronto_nbh_raw_data.merge(df_lat_lng, how = 'inner', on = 'Postal Code')
df_toronto_nbh_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<hr><h2>End of section2</h2><hr>

<h2><b>----Section 3: Clustering the toronto neighbourhoods.----</b></h2>

<h3>Section 3 Covers following points of peer graded assignment</h3>
<ul>
<li>1. Filter out neighbourhood data only for Borough with 'Toronto' in its name</li>
<li>2. For visualization use Folium to display toronto neighbourhoods BEFORE clustering </li>
<li>3. Foursquare API is used to get the venues for each neighbourhood</li>
<li>4. Venue category names are mapped to each neighbourhood it belongs to</li>
<li>5. Category names are used as a similarity OR dissimilarity measure</li>
<li>6. K-Means clustering algorithm is applied to neighbourhood data having categories for each neighbourhood</li>
<li>7. Best k value is chosen using elbow function</li>
<li>8. Clustered neighbourhoods are displayed in Folium map AFTER clustering</li>
</ul>

In [11]:
df_toronto = pd.DataFrame(df_toronto_nbh_data[df_toronto_nbh_data['Borough'].str.contains('Toronto')])
df_toronto.reset_index(drop = True, inplace=True)

In [12]:
#Import necessary libraries and dependencies
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import requests
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt
import matplotlib
import random
from colormap import rgb2hex

In [13]:
#get lat and lng for Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude

In [14]:
#Reduce Latitude and Longitude column values to float for using it in Folium map API
df_toronto.loc[:,('Latitude')] = pd.to_numeric(df_toronto.loc[:,('Latitude')],downcast='float')
df_toronto.loc[:,('Longitude')] = pd.to_numeric(df_toronto.loc[:,('Longitude')], downcast='float')

In [15]:
#Display folium map of Toronto with the neighbourhoods in df_toronto dataframe

map_toronto = folium.Map(location=[lat,lng], zoom_start=11)

for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'],df_toronto['Neighbourhood']):
    label = '{},{}'.format(neighbourhood,borough)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = 'green',
        fill_opacity = '0.7',
        parse_html = False
    ).add_to(map_toronto)
    
map_toronto

In [16]:
#Set Four square credentials 
CLIENT_ID='CMK42VYK5QGAICWXQZNMF2U4SH0VAU3AN05OZJMZK3WI5NWQ'
CLIENT_SECRET='XG1WH1D0GXKMEHTBRZW3I5OGGIGIKORBA3CPLO50BUUYB1ZZ'

In [17]:
#set common variable values.
radius = 500
limit=10
version = 20203112

df_toronto_venues = pd.DataFrame(columns=['Neighbourhood','Category','Latitude','Longitude'])

count = 1

#Use Foursquare API to get venue categories for each neighbourhood
for lat, lng, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&radius={}&limit={}&ll={},{}&v={}'.format(CLIENT_ID,CLIENT_SECRET,radius,limit,lat,lng,version)
    result = requests.get(url).json()
    
    neigh = neighbourhood
    latitude = lat
    longitude = lng
    list_category = []
    list_neigh = []
    list_lat = []
    list_lng = []
    
    
    for count in range(limit):
        if((result.get('response').get('venues')[count].get('categories') == [])):
            #print(d.get('response').get('venues')[count].get('categories'))
            category_name = 'Not assigned'
        else:
            category_name = result.get('response').get('venues')[count].get('categories')[0].get('name')
        list_category.append(category_name)
        list_neigh.append(neighbourhood)
        list_lat.append(lat)
        list_lng.append(lng)
        
    neighbourhood_categories = pd.DataFrame([list_neigh, list_category, list_lat, list_lng]).transpose()
    neighbourhood_categories.columns = ['Neighbourhood','Category','Latitude','Longitude']
    #New row with one postal code and multiple neighbourhoods is added to the dataframe.
    df_toronto_venues = df_toronto_venues.append(neighbourhood_categories, ignore_index=True)
        
df_toronto_venues.head()

,Neighbourhood,Category,Latitude,Longitude
0,"Regent Park, Harbourfront",Café,43.6543,-79.3606
1,"Regent Park, Harbourfront",Park,43.6543,-79.3606
2,"Regent Park, Harbourfront",Spa,43.6543,-79.3606
3,"Regent Park, Harbourfront",Coffee Shop,43.6543,-79.3606
4,"Regent Park, Harbourfront",Gourmet Shop,43.6543,-79.3606


In [18]:
df_toronto_venues_data_wrangling = pd.DataFrame(df_toronto_venues)

#Drop rows with venues having 'Not assigned' Category names
df_toronto_venues_data_wrangling.drop(index=df_toronto_venues_data_wrangling[df_toronto_venues_data_wrangling['Category']=='Not assigned'].index.values,inplace=True)
df_toronto_venues_data_wrangling.reset_index(inplace=True, drop=True)

In [19]:
#Get one hot encoding using Category names for neighbourhood
toronto_venues_onehot = pd.get_dummies(df_toronto_venues_data_wrangling[['Category']], prefix="", prefix_sep="")
toronto_venues_onehot['Neighbourhood'] = df_toronto_venues_data_wrangling['Neighbourhood']

In [20]:
#rearrange dataframe columns for better visibility.
col_count = toronto_venues_onehot.shape[1]
fixed_cols = [toronto_venues_onehot.columns[-1]] + list(toronto_venues_onehot.columns[0:col_count-1])
toronto_venues_onehot = toronto_venues_onehot[fixed_cols]
toronto_venues_onehot.head()

,Neighbourhood,ATM,Adult Boutique,Airport,Alternative Healer,American Restaurant,Antique Shop,Art Gallery,Assisted Living,Auditorium,...,Tech Startup,Thai Restaurant,Theme Park,Thrift / Vintage Store,Trail,Train Station,Tunnel,University,Vietnamese Restaurant,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#group the rows by Neighbourhood and calculate mean for each group for each category value
toronto_venue_group_mean = toronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()

#remove Neighbourhood column and prepare new dataframe for clustering processing
toronto_venue_group_clustering = toronto_venue_group_mean.drop('Neighbourhood',axis=1)
toronto_venue_group_clustering.head()

,ATM,Adult Boutique,Airport,Alternative Healer,American Restaurant,Antique Shop,Art Gallery,Assisted Living,Auditorium,Auto Workshop,...,Tech Startup,Thai Restaurant,Theme Park,Thrift / Vintage Store,Trail,Train Station,Tunnel,University,Vietnamese Restaurant,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#Use K-means to find out the clusters in the data.
#To choose an optimal value of k we use elbow method.

distortions = []
inertias = []

#let's run k means using multiple values of k from 1 to 15
klimit = 15
for k in range(1,klimit):
    clusters = KMeans(n_clusters=k, n_init=10, algorithm='auto').fit(toronto_venue_group_clustering)
    distortions.append(sum(np.min(cdist(toronto_venue_group_clustering, clusters.cluster_centers_, 
                      'euclidean'),axis=1)) / toronto_venue_group_clustering.shape[0])
    inertias.append(clusters.inertia_)


In [23]:
df_best_k = pd.DataFrame([range(1,15),distortions,inertias]).transpose()
df_best_k.columns=['k-value','distortion','inertias']
df_best_k #folloing table clearly shows us that elbow is most probably at k = 7 Thus best value of k = 7

,k-value,distortion,inertias
0,1.0,0.336076,4.453139
1,2.0,0.324159,4.149983
2,3.0,0.312822,3.871616
3,4.0,0.301500,3.671317
4,5.0,0.297384,3.520642
5,6.0,0.288530,3.386361
6,7.0,0.277053,3.217079
7,8.0,0.272936,3.041648
8,9.0,0.265969,2.892413
9,10.0,0.250027,2.748345


In [24]:
#Running K means with K = 7 to get the cluster values for neighbours.
k = 7
clusters = KMeans(n_clusters=k, n_init=10, algorithm='auto').fit(toronto_venue_group_clustering)
clusters.labels_

array([6, 5, 4, 6, 2, 1, 2, 5, 2, 3, 1, 1, 4, 0, 2, 6, 4, 1, 4, 2, 4, 0, 4,
       6, 2, 1, 3, 5, 4, 2, 2, 6, 2, 6, 6, 4, 1, 4, 0], dtype=int32)

In [25]:
toronto_neighbourhood_clusters = pd.DataFrame(columns=['Neighbourhood','Clusters'])
toronto_neighbourhood_clusters['Neighbourhood'] = toronto_venue_group_mean['Neighbourhood']
toronto_neighbourhood_clusters['Clusters'] = clusters.labels_
#df_toronto.loc[:,['Neighbourhood','Latitude','Longitude']]
toronto_neighbourhood_clusters = toronto_neighbourhood_clusters.merge(df_toronto.loc[:,['Neighbourhood','Latitude','Longitude','Borough']], how = 'inner', on = 'Neighbourhood')
toronto_neighbourhood_clusters.loc[:,('Clusters')] = pd.to_numeric(toronto_neighbourhood_clusters.loc[:,('Clusters')],downcast='integer')

In [26]:
#Display folium map of CLUSTERED Toronto neighbourhoods
#Get lat and lng for Toronto

address = 'Toronto, CA'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude

clustered_map_toronto = folium.Map(location=[toronto_lat,toronto_lng], zoom_start=11)

clr = []
for i in range(0,k):
    cl = rgb2hex(random.randint(1,255),random.randint(1,255),random.randint(1,255))
    clr.append(cl)
    

for lat, lng, borough, neighbourhood,clstr in zip(toronto_neighbourhood_clusters['Latitude'], toronto_neighbourhood_clusters['Longitude'], toronto_neighbourhood_clusters['Borough'],toronto_neighbourhood_clusters['Neighbourhood'],toronto_neighbourhood_clusters['Clusters']):
    label = '{},{}'.format(neighbourhood,borough)
    label = folium.Popup(label,parse_html=True)
    
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = clr[clstr],
        fill = True,
        fill_opacity = '0.7',
        parse_html = False
    ).add_to(clustered_map_toronto)
    
clustered_map_toronto